In [1]:
import pandas as pd
import json
import sqlite3

In [ ]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
with open("users.json", "r") as file:
    users_data = json.load(file)
users = pd.DataFrame(users_data)
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as file:
    sql_script = file.read()
    conn.executescript(sql_script)
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
merged_df = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
merged_df = pd.merge(
    merged_df,
    restaurants,
    on="restaurant_id",
    how="left"
)
merged_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
merged_df = merged_df.drop(columns=["restaurant_name_x"])
merged_df = merged_df.rename(
    columns={"restaurant_name_y": "restaurant_name"}
)
merged_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'name', 'city', 'membership', 'restaurant_name', 'cuisine', 'rating'],
      dtype='object')

In [7]:
merged_df["order_date"] = pd.to_datetime(
    merged_df["order_date"],
    format="%d-%m-%Y"
)

merged_df.dtypes


order_id                    int64
user_id                     int64
restaurant_id               int64
order_date         datetime64[ns]
total_amount              float64
name                       object
city                       object
membership                 object
restaurant_name            object
cuisine                    object
rating                    float64
dtype: object

In [8]:
merged_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("✅ Final dataset saved as final_food_delivery_dataset.csv")


✅ Final dataset saved as final_food_delivery_dataset.csv
